In [1]:
import pandas as pd

df = pd.read_csv("data/preprocessed.csv")


/tmp/ipykernel_53737/3777639536.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df

,news_desc,category_id
0,a million american roll sleev omicron target c...,0
1,american airlin flyer charg ban life punch fli...,0
2,woman call cop black birdwatch lose lawsuit em...,0
3,cleaner dead bell bathroom a day bodi found po...,0
4,report get ador surpri boyfriend live to behin...,0
...,...,...
129537,local mon give hand singl mother tricia ward o...,37
129538,father challeng jail sentenc child support lan...,37
129539,tip help dog deal divorc break hard famili pet...,37
129540,find love advic divorc woman legal freedom com...,37


In [6]:
df.dropna(inplace=True)



In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create an instance of TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.85, max_features=8000, min_df=0.1, use_idf=True, ngram_range=(1,3))

# Fit and transform the text data
tfidf_matrix = tfidf_vectorizer.fit_transform(df['news_desc'])

# Convert the TF-IDF matrix to a dense matrix
tfidf_matrix_dense = tfidf_matrix.todense()

# Print the TF-IDF matrix
print(tfidf_matrix_dense)


[[0. 1.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]
